# Link Demo

We demo 2 different functions for collecting link data from Wikipedia:
1. `get_links` - for collecting different types of links from a list of pages.
2. `pipeline_get_links` - A convenience wrapper function that, in addition to the above, also sets up session and redirect maps.

## Setup

In [1]:
import wikitoolkit
import pandas as pd

my_agent = 'mwapi testing <p.gildersleve@lse.ac.uk>'
wtsession = wikitoolkit.WTSession('en.wikipedia', user_agent=my_agent)
toparts = pd.read_csv('data/topviews-2024_07_31.csv')
artlist = toparts['Page'].unique().tolist() # ~1000 top articles yesterday


## `get_links`

This can be used to collect in-, out-, interwiki-, lang-, and external links to/from a list of articles.

### Out-links

In [2]:
outlinks = await wikitoolkit.get_links(wtsession, mode='out', titles=artlist[:10])
edges = pd.DataFrame(set([(k, x['title']) for k, v in outlinks.items() for x in v]),
            columns=['source', 'target'])
edges

Getting out-links


,source,target
0,Michael Phelps,Dan Veatch
1,Michael Phelps,Malcolm Champion
2,Kamala Harris,Deb Haaland
3,Kamala Harris,Choctaw freedmen
4,Sunisa Lee,Michigan Wolverines women's gymnastics
...,...,...
9770,2024 Summer Olympics,Papua New Guinea at the 2024 Summer Olympics
9771,Kamala Harris,Governor of Minnesota
9772,Deadpool & Wolverine,Jemma Simmons
9773,Simone Biles,The New York Times


### External links

In [3]:
extlinks = await wikitoolkit.get_links(wtsession, mode=['ext'], titles=artlist[:10])
edges = pd.DataFrame([{'source': k, 'target': x['url']}for k, v in extlinks.items()
                      for x in v], columns=['source', 'target'])
edges

Getting ext-links


,source,target
0,Kamala Harris,https://www.mercurynews.com/2020/08/18/heres-k...
1,Kamala Harris,https://web.archive.org/web/20201014130548/htt...
2,Kamala Harris,https://www.harris.senate.gov/about
3,Kamala Harris,https://abcnews.go.com/Politics/kamala-harris-...
4,Kamala Harris,https://www.washingtonpost.com/politics/kamala...
...,...,...
3495,Jonathan Owens,https://www.nbcnewyork.com/news/sports/nfl/sim...
3496,Jonathan Owens,https://www.usatoday.com/story/sports/nfl/2024...
3497,Jonathan Owens,https://www.espn.com/nfl/story?id=41179488&_sl...
3498,Jonathan Owens,https://archive.today/20240909185841/https://w...


### All kinds of link

In [4]:
alllinks = await wikitoolkit.get_links(wtsession, mode=['in', 'out', 'lang',
                                                          'interwiki', 'ext'],
                                     titles=artlist[:10])

# Create a dataframe for each type of link
wikiedges = pd.DataFrame(set([(k, x['title']) for k, v in alllinks['out'].items()
                              for x in v] +
                             [(x['title'], k) for k, v in alllinks['in'].items()
                              for x in v]), columns=['source', 'target'])
extedges = pd.DataFrame([{'source': k, 'target': x['url']}
                         for k, v in alllinks['ext'].items() for x in v])
interwikiedges = pd.concat({k: pd.DataFrame(v) for k, v in alllinks['interwiki'].items()}
                            ).reset_index(drop=True, level=1).reset_index(
                            ).rename(columns={'index': 'source',
                                              'prefix': 'target_prefix',
                                              'title': 'target_title'})
langedges = pd.concat({k: pd.DataFrame(v).T for k, v in alllinks['lang'].items()}
                     ).reset_index().rename(columns={'level_0': 'source',
                                                     'level_1': 'target_lang',
                                                     0: 'target_title'})

display(wikiedges.head(), extedges.head(), interwikiedges.head(), langedges.head())

Getting in-links
Getting out-links
Getting lang-links
Getting interwiki-links
Getting ext-links


,source,target
0,Michael Phelps,Dan Veatch
1,Sunisa Lee,Michigan Wolverines women's gymnastics
2,Katie Ledecky,Susie O'Neill
3,Deadpool & Wolverine,Black Talon (comics)
4,List of career achievements by Roger Federer,Michael Phelps


,source,target
0,Kamala Harris,https://www.mercurynews.com/2020/08/18/heres-k...
1,Kamala Harris,https://web.archive.org/web/20201014130548/htt...
2,Kamala Harris,https://www.harris.senate.gov/about
3,Kamala Harris,https://abcnews.go.com/Politics/kamala-harris-...
4,Kamala Harris,https://www.washingtonpost.com/politics/kamala...


,source,target_prefix,target_title
0,Kamala Harris,c,Category:Kamala_Harris
1,Kamala Harris,commons,Category:Kamala_Harris
2,Kamala Harris,commons,Vice_President_of_the_United_States
3,Kamala Harris,d,Q10853588
4,Kamala Harris,n,Category:Kamala_Harris


,source,target_lang,target_title
0,Kamala Harris,af,Kamala Harris
1,Kamala Harris,an,Kamala Harris
2,Kamala Harris,ar,كامالا هاريس
3,Kamala Harris,arz,كامالا هاريس
4,Kamala Harris,as,কমলা হেৰিছ


## `pipeline_get_links`

This function sets up the session, fixes redirects with PageMaps (see `redirectsdemo.ipynb`), and collects link data. It is a convenience function that wraps the previous function. Note that this does not require manual setup of the `wtsession`.

In [5]:
links, pagemaps = await wikitoolkit.pipeline_get_links('en.wikipedia', my_agent,
                                                titles=artlist[:10],
                                                gl_args={'mode':['out', 'in'],
                                                        'update_maps':True})

# Get list of articles and edges (excluding redlinks, those with no page id)
articles = set([x['title'] for y in links['in'].values() for x in y] +
                [x['title'] for y in links['out'].values() for x in y
                 if pagemaps.id_map[x['title']] != -1])
edges = pd.DataFrame(set([(k, x['title']) for k, v in links['out'].items()
                          for x in v if pagemaps.id_map[x['title']] != -1] +
                         [(x['title'], k) for k, v in links['in'].items()
                          for x in v]), columns=['source', 'target'])

# Additionally returns new pagemaps object (if not supplied), storing redirects, normalizations, and page ids
# It's recommended to create a single pagemaps object in a project and update it with each call
print(pagemaps)

print(len(articles), len(edges))
display(edges.head())


Getting out-links
Getting in-links
Redirects: 1117, Norms: 0, IDs: 18971, Existing: 0
17854 26079


,source,target
0,Michael Phelps,Dan Veatch
1,Sunisa Lee,Michigan Wolverines women's gymnastics
2,Katie Ledecky,Susie O'Neill
3,Deadpool & Wolverine,Black Talon (comics)
4,List of career achievements by Roger Federer,Michael Phelps


In [6]:
# testing with page ids instead of titles
ids = [pagemaps.id_map[x] for x in artlist[:10]]
# supply pre-existing pagemaps object
links = await wikitoolkit.pipeline_get_links('en.wikipedia', my_agent,
                                                pageids=ids, pagemaps=pagemaps,
                                                gl_args={'mode':['out', 'in',
                                                                 'lang', 'ext',
                                                                 'interwiki'],
                                                        'update_maps':True})
print(pagemaps)
print(len(links['in']), len(links['out']), len(links['lang']),
        len(links['ext']), len(links['interwiki']))

Getting out-links
Getting in-links
Getting lang-links
Getting ext-links
Getting interwiki-links
Redirects: 1117, Norms: 0, IDs: 18971, Existing: 0
10 10 10 10 10


### Redirect demo

The pipeline function can also fix redirects / normalize titles as part of the process. The redirect has to be recorded on Wikipedia, it can't magically fix typos.

In [7]:
bad_titles = [x.lower() for x in artlist[:10]] + ['thisisnotatitle']
links = await wikitoolkit.pipeline_get_links('en.wikipedia', my_agent,
                                                titles=bad_titles, pagemaps=pagemaps,
                                                gl_args={'mode':['out', 'in'],
                                                        'update_maps':True})

# Get list of articles and edges (excluding redlinks, those with no page id)
articles = set([x['title'] for y in links['in'].values() for x in y] +
                [x['title'] for y in links['out'].values() for x in y
                 if pagemaps.id_map[x['title']] != -1])
edges = pd.DataFrame(set([(k, x['title']) for k, v in links['out'].items()
                          for x in v if pagemaps.id_map[x['title']] != -1] +
                         [(x['title'], k) for k, v in links['in'].items()
                          for x in v]), columns=['source', 'target'])

# has updated pagemaps object inplace with norms/redirects/pageids
print(pagemaps)

print(len(articles), len(edges))
display(edges.head())

Getting out-links
Getting in-links
Redirects: 1125, Norms: 10, IDs: 18979, Existing: 0
7246 10134


,source,target
0,Qualified New York political parties,Kamala Harris
1,Michael Phelps,Dan Veatch
2,Michael Phelps,Malcolm Champion
3,Kamala Harris,Deb Haaland
4,Kamala Harris,Choctaw freedmen


## Large scale collection

The functions can be used to collect data on a large scale asyncronously, with adaptations around any rate limiting. Here we demonstrate collecting data on 1000 random articles. In-link collection is particularly slow for popular articles as so many other articles link to them.

In [8]:
links, pagemaps = await wikitoolkit.pipeline_get_links('en.wikipedia', my_agent,
                                                titles=artlist,
                                                gl_args={'mode':['out', 'in'],
                                                        'update_maps':True})

# Get list of articles and edges (excluding redlinks, those with no page id)
articles = set([x['title'] for y in links['in'].values() for x in y] +
                [x['title'] for y in links['out'].values() for x in y
                 if pagemaps.id_map[x['title']] != -1])
edges = pd.DataFrame(set([(k, x['title']) for k, v in links['out'].items()
                          for x in v if pagemaps.id_map[x['title']] != -1] +
                         [(x['title'], k) for k, v in links['in'].items()
                          for x in v]), columns=['source', 'target'])

print(len(articles), len(edges))
display(edges.head())

Getting out-links
('MediaWiki returned an error:', 'Could not decode as JSON:\n<!DOCTYPE html>\n<html lang="en">\n<meta charset="utf-8">\n<title>Wikimedia Error</title>\n<style>\n* { margin: 0; padding: 0; }\nbody { background: #fff; font: 15px/1.6 sans-serif; color: #333; }\n.content { margin: 7% auto 0; padding: 2em 1em 1em; max-width: 640px; }\n.footer { clear: both; margin-top: 14%; border-top: 1px solid #e5e5e5; background: #f9f')
80.97% complete
Trying again at n=800 with batchsize=100
Increasing batchsize to 200
Getting in-links
Cannot connect to host en.wikipedia.org:443 ssl:default [Connection reset by peer]
20.24% complete
Trying again at n=200 with batchsize=100
Increasing batchsize to 200
Cannot connect to host en.wikipedia.org:443 ssl:default [Connection reset by peer]
50.61% complete
Trying again at n=500 with batchsize=100
Cannot connect to host en.wikipedia.org:443 ssl:default [Connection reset by peer]
50.61% complete
Trying again at n=500 with batchsize=50
Increasing 

,source,target
0,2019 Reign FC season,Twitter
1,Susyana Tjhan,Indonesia
2,Khamoshiyan,Twitter
3,Seven Tour,Germany
4,List of magicians in film,Hugh Jackman


In [9]:
# I have a hunch that ordering the articles by number of (in-)links
# will make collection of articles more efficient for subsequent API calls.
# Let's try it

top_indegree = edges['target'].value_counts()
ordered_artlist = ([x for x in top_indegree.index if x in artlist] +
                   [x for x in artlist if x not in top_indegree.index])[::-1]
ordered_artlist[-10:] # These articles have loooots of in-links, v slow to collect

new_links, new_pagemaps = await wikitoolkit.pipeline_get_links('en.wikipedia', my_agent,
                                                titles=ordered_artlist,
                                                gl_args={'mode':['out', 'in'],
                                                        'update_maps':True})

# Get list of articles and edges (excluding redlinks, those with no page id)
new_articles = set([x['title'] for y in new_links['in'].values() for x in y] +
                [x['title'] for y in new_links['out'].values() for x in y
                 if new_pagemaps.id_map[x['title']] != -1])
new_edges = set([(k, x['title']) for k, v in new_links['out'].items() 
                 for x in v if new_pagemaps.id_map[x['title']] != -1] +
                [(x['title'], k) for k, v in new_links['in'].items()
                for x in v])

print(len(new_articles), len(new_edges))

Getting out-links
('MediaWiki returned an error:', 'Could not decode as JSON:\n<!DOCTYPE html>\n<html lang="en">\n<meta charset="utf-8">\n<title>Wikimedia Error</title>\n<style>\n* { margin: 0; padding: 0; }\nbody { background: #fff; font: 15px/1.6 sans-serif; color: #333; }\n.content { margin: 7% auto 0; padding: 2em 1em 1em; max-width: 640px; }\n.footer { clear: both; margin-top: 14%; border-top: 1px solid #e5e5e5; background: #f9f')
80.97% complete
Trying again at n=800 with batchsize=100
Increasing batchsize to 200
Getting in-links
('MediaWiki returned an error:', 'Could not decode as JSON:\n<!DOCTYPE html>\n<html lang="en">\n<meta charset="utf-8">\n<title>Wikimedia Error</title>\n<style>\n* { margin: 0; padding: 0; }\nbody { background: #fff; font: 15px/1.6 sans-serif; color: #333; }\n.content { margin: 7% auto 0; padding: 2em 1em 1em; max-width: 640px; }\n.footer { clear: both; margin-top: 14%; border-top: 1px solid #e5e5e5; background: #f9f')
40.49% complete
Trying again at n=40